<a href="https://colab.research.google.com/github/YaninaK/anomaly-detection/blob/b1/notebooks/01_EDA_%26_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ аномалий в начислениях за тепловую энергию


## 1. Анализ и подготовка данных



In [1]:
initiate = False
if initiate:
  !git init -q
  !git clone -b b1  https://github.com/YaninaK/anomaly-detection.git -q

  from google.colab import drive
  drive.mount('/content/drive')

  !unzip -u -q /content/drive/MyDrive/ML_projects/08_anomaly_detection/data/01_raw/task#3.zip -d /content/anomaly-detection/data/01_raw

%cd /content/anomaly-detection/notebooks

/content/anomaly-detection/notebooks


In [2]:
import os
import sys

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "..", "src", "anomaly_detection"))

In [3]:
import datetime

import numpy as np
import pandas as pd

from data.make_dataset import load_data
from data.preprocess import Preprocess

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
PATH = "/content/anomaly-detection/"

## 1. Чтение данных

In [6]:
folder_path = '../data/01_raw/'
file_names = os.listdir(folder_path)

file_names

['Ноябрь 2022 Уфа.xlsx',
 'Ноябрь 2021 Уфа.xls',
 'Сентябрь 2021 Уфа.xls',
 'Примеры аномалий.docx',
 'Февраль 2023 Уфа.xlsx',
 'Март 2022 Уфа.xls',
 'Июнь 2023 Уфа.xlsx',
 'Апрель 2022 Уфа.xls',
 'Декабрь 2022 Уфа.xlsx',
 'Декабрь 2021 Уфа.xls',
 'Февраль 2022 Уфа.xls',
 'Температуры, продолжительность ОП.xls',
 'Май 2022 Уфа.xls',
 'Август 2022 Уфа.xlsx',
 'Октябрь 2021 Уфа.xls',
 'Октябрь 2022 Уфа.xlsx',
 'Июль 2022 Уфа.xlsx',
 '.gitkeep',
 'Июль 2021 Уфа.xls',
 'Август 2021 Уфа.xls',
 'Март 2023 Уфа.xlsx',
 'Май 2023 Уфа.xlsx',
 'Январь 2023 Уфа.xlsx',
 'Январь 2022 Уфа.xls',
 'Апрель 2023 Уфа.xlsx',
 'Июнь 2022 Уфа.xls',
 'Тип строения, этажность, площадь, год постройки.xlsx',
 'Сентябрь 2022 Уфа.xlsx']

In [7]:
regenerate = False
data, temperature, buildings = load_data(folder_path, regenerate, path=PATH)

100%|██████████| 24/24 [00:15<00:00,  1.54it/s]


In [8]:
print(f"data.shape = {data.shape}\n")
data.sample(2)

data.shape = (75391, 9)



,index,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",Период потребления
55942,2426,Уфа,103539,NaN,"г Уфа, ул. Максима Рыльского, д.2",Многоквартирный дом,2023-04-01,146.113,2023-03-01
22094,180,Уфа,107301,NaN,"г Уфа, ул. Менделеева, д.201 корп.1",Многоквартирный дом,2022-05-01,79.174,2022-04-01


In [9]:
data["Адрес объекта"].nunique()

4278

* Тип строения, этажность, площадь, год постройки

In [10]:
print(f"buildings.shape = {buildings.shape}\n")
buildings.sample(2)

buildings.shape = (5873, 5)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
5206,"г Уфа, ул. Ульяновых, д.67",Другое строение,2,NaN,0.01
3986,"г Уфа, ул. Пекинская, д.33 корп.1",Многоквартирный дом,9,1987-01-01 00:00:00,5647.38


In [11]:
buildings["Адрес объекта"].nunique()

5695

* Температуры, продолжительность отопительного периода.

  * ОЗП — осенне-зимний период, время, когда работает система отопления.

In [12]:
temperature = pd.read_excel(
    f'{folder_path}Температуры, продолжительность ОП.xls', index_col=1, skiprows=1
).T[1:].reset_index()

temperature

Период,index,"Тн.в, град.С","Продолжительность ОЗП, сут."
0,2021-07-01,NaN,NaN
1,2021-08-01,NaN,NaN
2,2021-09-01,NaN,NaN
3,2021-10-01,6.696552,29.0
4,2021-11-01,-1.000000,31.0
5,2021-12-01,-5.176667,30.0
6,2022-01-01,-11.500000,31.0
7,2022-02-01,-6.735714,28.0
8,2022-03-01,-5.151613,31.0
9,2022-04-01,5.650000,34.0


## 2. Анлиз и подготовка данных

In [13]:
preprocess = Preprocess()
data, buildings = preprocess.fit_transform(data, buildings)

### 2.1. Объекты с Общей площадью <= 0.1

In [14]:
cond1 = buildings["Тип Объекта"] == "Многоквартирный дом"
cond2 = buildings["Общая площадь объекта"] <= 0.1

df = buildings[cond1 & cond2].iloc[:, :-1]
print(f"Число объектов без данных о площади: {df.shape[0]}")

df.head(2)

Число объектов без данных о площади: 25


,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
949,"г Уфа, ул. Айская",Многоквартирный дом,20,NaT,0.01
1452,"г Уфа, ул. Большая Гражданская, д.49",Многоквартирный дом,1,2018-11-01,0.02


* У 25 многоквартирных домов Общая площадь объекта не превышает 0.1.

### 2.2. Объекты без Даты постройки

In [15]:
cond3 = buildings["Дата постройки"].isnull()
df = buildings[cond1 & cond3].iloc[:, :-1]

print(f"Число объектов без данных о дате постройки: {df.shape[0]}")

df.head(2)

Число объектов без данных о дате постройки: 34


,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
62,"г Уфа, б-р. Ибрагимова, д.90 корп.2",Многоквартирный дом,20,NaT,53759.1
836,"г Уфа, ул. 50-летия Октября, д.3 корп.П",Многоквартирный дом,5,NaT,279.6


У 34 многоквартирных дома нет признака Дата постройки.

### 2.3. Неуникальные адреса многоквартирных домов

In [16]:
cond1_a = data["Тип объекта"] == "Многоквартирный дом"
cond1_b = buildings["Тип Объекта"] == "Многоквартирный дом"

In [17]:
df = buildings[cond1_b & buildings[cond1_b].duplicated(subset="Адрес объекта", keep=False)]
df

,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2
1583,"г Уфа, ул. Вологодская, д.13",Многоквартирный дом,9,1979-01-01,2022.00,"г Уфа, ул. Вологодская, д.13, Подобъект №984990"
1584,"г Уфа, ул. Вологодская, д.13",Многоквартирный дом,9,1979-01-01,8256.50,"г Уфа, ул. Вологодская, д.13"
1603,"г Уфа, ул. Вологодская, д.20",Многоквартирный дом,12,1978-01-01,655.00,"г Уфа, ул. Вологодская, д.20, extra"
1604,"г Уфа, ул. Вологодская, д.20",Многоквартирный дом,12,1978-01-01,3681.40,"г Уфа, ул. Вологодская, д.20"
2101,"г Уфа, ул. Интернациональная, д.113",Многоквартирный дом,1,1964-01-01,1161.90,"г Уфа, ул. Интернациональная, д.113, extra"
2102,"г Уфа, ул. Интернациональная, д.113",Многоквартирный дом,5,1964-01-01,2563.70,"г Уфа, ул. Интернациональная, д.113"
2397,"г Уфа, ул. Кирова, д.95",Многоквартирный дом,1,1971-01-01,0.01,"г Уфа, ул. Кирова, д.95, Подобъект №46590"
2398,"г Уфа, ул. Кирова, д.95",Многоквартирный дом,5,1971-01-01,3448.80,"г Уфа, ул. Кирова, д.95"
3765,"г Уфа, ул. Нежинская, д.6",Многоквартирный дом,9,1972-01-01,427.70,"г Уфа, ул. Нежинская, д.6, extra"
3766,"г Уфа, ул. Нежинская, д.6",Многоквартирный дом,9,1972-01-01,2544.60,"г Уфа, ул. Нежинская, д.6"


* В базе многоквартирных домов 14 объектов с неуникальными адресами - по 2 на каждый адрес:
  * ```г Уфа, ул. Вологодская, д.13```,
  * ```г Уфа, ул. Вологодская, д.20```,
  * ```г Уфа, ул. Интернациональная, д.113```,
  * ```г Уфа, ул. Кирова, д.95```,
  * ```г Уфа, ул. Нежинская, д.6```,
  * ```г Уфа, ул. Революционная, д.88```,
  * ```г Уфа, ул. Энтузиастов, д.6```.

  * На 3 объекта из 14 по адресам ниже из них не выставляются счета:
    * ```г Уфа, ул. Вологодская, д.20```,
    * ```г Уфа, ул. Интернациональная, д.113```,
    * ```г Уфа, ул. Нежинская, д.6```.


### 2.4. Подобъекты

In [18]:
addr = sorted(list(set(data[cond1_a]["Адрес объекта"]) - set(buildings[cond1_b]["Адрес объекта"])))
addr

['г Уфа, б-р. Давлеткильдеева, д.18, Подобъект №987152',
 'г Уфа, б-р. Давлеткильдеева, д.5, Подобъект №985759',
 'г Уфа, б-р. Ибрагимова, д.34, Подобъект №983365',
 'г Уфа, б-р. Молодежный, д.2, Подобъект №992132',
 'г Уфа, б-р. Молодежный, д.6 корп.1, Подобъект №983457',
 'г Уфа, б-р. Тухвата Янаби, д.22, Подобъект №351681',
 'г Уфа, пр-кт Октября, д.122, Подобъект №42688',
 'г Уфа, пр-кт Октября, д.174/2, Подобъект №985197',
 'г Уфа, пр-кт Октября, д.25, Подобъект №48042',
 'г Уфа, пр-кт Октября, д.25, Подобъект №48043',
 'г Уфа, пр-кт Октября, д.25, Подобъект №48044',
 'г Уфа, пр-кт Октября, д.60 корп.1, Подобъект №51211',
 'г Уфа, пр-кт Октября, д.78 корп.3, Подобъект №989097',
 'г Уфа, ул. 50 лет СССР, д.2, Подобъект №47840',
 'г Уфа, ул. 8 Марта, д.12 корп.3, Подобъект №49931',
 'г Уфа, ул. Академика Королева, д.2 корп.2, Подобъект №51212',
 'г Уфа, ул. Ахметова, д.324, Подобъект №986621',
 'г Уфа, ул. Баязита Бикбая, д.44, Подобъект №983215',
 'г Уфа, ул. Блюхера, д.17, Подобъе

In [19]:
len(addr)

77

In [20]:
s = 'г Уфа, ул. Энтузиастов, д.14'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Энтузиастов, д.14'
 'г Уфа, ул. Энтузиастов, д.14, Подобъект №984765'
 'г Уфа, ул. Энтузиастов, д.14, Подобъект №984513'
 'г Уфа, ул. Энтузиастов, д.14, Подобъект №984848']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
5715,"г Уфа, ул. Энтузиастов, д.14",Многоквартирный дом,18,2018-01-01,27686.6


In [21]:
data[data["Адрес объекта"]=="г Уфа, ул. Энтузиастов, д.14"]["№ ОДПУ"].unique()

array(['278553,278511'], dtype=object)

In [22]:
data[data["Адрес объекта"]=="г Уфа, ул. Энтузиастов, д.14, Подобъект №984765"]["№ ОДПУ"].unique()

array(['18480739'], dtype=object)

* У подобъектов и объектов отличается ```№ ОДПУ```- учет ведется раздельно, при этом площадь - общая.
* Для использования в моделях необходимо включить подобъекты в объекты, при выдаче результата - оценить площадь подобъеков и объектов в разрезе ```№ ОДПУ```.

In [23]:
sorted(list(set(data[cond1_a]["Адрес объекта 2"]) - set(buildings[cond1_b]["Адрес объекта 2"])))

['г Уфа, ул. Пархоменко, д.156 корп.Литер 2в',
 'г Уфа, ул. Уфимское шоссе, д.4']

### 2.5. Многоквартирные дома, отсутствующие в базе данных объектов

#### 2.5.1 ```г Уфа, ул. Пархоменко, д.156 корп.Литер 2в```

In [24]:
s = 'г Уфа, ул. Пархоменко, д.156'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Пархоменко, д.156 корп.б'
 'г Уфа, ул. Пархоменко, д.156 корп.в'
 'г Уфа, ул. Пархоменко, д.156 корп.Литер 2в']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
3937,"г Уфа, ул. Пархоменко, д.156 корп.А",Многоквартирный дом,25,2021-11-01,10428.70
3938,"г Уфа, ул. Пархоменко, д.156 корп.б",Многоквартирный дом,16,2019-11-01,21415.40
3939,"г Уфа, ул. Пархоменко, д.156 корп.в",Многоквартирный дом,25,2019-11-01,11241.00
3940,"г Уфа, ул. Пархоменко, д.156 корп.г",Многоквартирный дом,25,2022-11-01,10515.70
3941,"г Уфа, ул. Пархоменко, д.156 корп.Д",Многоквартирный дом,16,2021-11-01,9040.71
3942,"г Уфа, ул. Пархоменко, д.156 корп.е",Многоквартирный дом,25,NaT,0.01
3943,"г Уфа, ул. Пархоменко, д.156 корп.ж",Многоквартирный дом,25,NaT,0.10


* ```г Уфа, ул. Пархоменко, д.156 корп.Литер 2в``` в ```data``` не соответствует
аналогу в ```buildings```:
  * ```г Уфа, ул. Пархоменко, д.156 корп.А```
  * ```г Уфа, ул. Пархоменко, д.156 корп.г```
  * ```г Уфа, ул. Пархоменко, д.156 корп.Д```

#### 2.5.2 ```г Уфа, ул. Уфимское шоссе, д.4```

In [25]:
s = 'г Уфа, ул. Уфимское шоссе, д.4'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Уфимское шоссе, д.4' 'г Уфа, ул. Уфимское шоссе, д.4 корп.1']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
5275,"г Уфа, ул. Уфимское шоссе, д.4 корп.1",Многоквартирный дом,9,1976-01-01,3821.3


In [26]:
data[data["Адрес объекта"]=="г Уфа, ул. Уфимское шоссе, д.4"]["№ ОДПУ"].unique()

array(['102522'], dtype=object)

In [27]:
data[data["Адрес объекта"]=="г Уфа, ул. Уфимское шоссе, д.4 корп.1"]["№ ОДПУ"].unique()

array(['104237'], dtype=object)

* Объекта ```г Уфа, ул. Уфимское шоссе, д.4``` нет. Есть только ```г Уфа, ул. Уфимское шоссе, д.4 корп.1```, но у него другой ```№ ОДПУ```.

#### 2.5.3. ```г Уфа, ул. Владивостокская, д.10 корп.3,4```

In [28]:
s = 'г Уфа, ул. Владивостокская, д.10'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Владивостокская, д.10'
 'г Уфа, ул. Владивостокская, д.10 корп.3,4']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
1554,"г Уфа, ул. Владивостокская, д.10",Многоквартирный дом,14,2015-01-01,23745.65


In [29]:
data[data["Адрес объекта"]=="г Уфа, ул. Владивостокская, д.10"]["№ ОДПУ"].unique()

array(['248788/15024285/20101973/23-190725', '20-101973'], dtype=object)

In [30]:
data[data["Адрес объекта"]=="г Уфа, ул. Владивостокская, д.10 корп.3,4"]["№ ОДПУ"].unique()

array(['15024285'], dtype=object)

In [31]:
data[data["Адрес объекта"]=="г Уфа, ул. Владивостокская, д.10 корп.3,4"].iloc[:, :-1]

,index,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",Период потребления
4516,97,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2021-11-01
8755,22,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2021-09-01
13227,1277,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2023-02-01
16438,55,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2022-03-01
22012,98,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2022-04-01
30834,118,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2021-12-01
35116,61,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,2022-03-01,221.453,2022-02-01
39443,35,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2022-05-01
41967,139,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2021-10-01
51522,87,Уфа,15024285,NaN,"г Уфа, ул. Владивостокская, д.10 корп.3,4",Многоквартирный дом,NaT,NaN,2021-07-01


* Объект ```г Уфа, ул. Владивостокская, д.10 корп.3,4``` в data должен попасть в ```г Уфа, ул. Владивостокская, д.10``` - корп.3,4 избыточен.

#### 2.5.4 ``` г Уфа, ул. Даута Юлтыя, д.12```

In [32]:
s = 'г Уфа, ул. Даута Юлтыя, д.12'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Даута Юлтыя, д.12'
 'г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20654'
 'г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20655']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
1841,"г Уфа, ул. Даута Юлтыя, д.12",Многоквартирный дом,16,2011-01-01,13465.5


In [33]:
data[data["Адрес объекта"]=="г Уфа, ул. Даута Юлтыя, д.12"]["№ ОДПУ"].unique()

array(['89612 89617'], dtype=object)

In [34]:
data[data["Адрес объекта"]=="г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20654"]["№ ОДПУ"].unique()

array(['89612'], dtype=object)

In [35]:
data[data["Адрес объекта"]=="г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20655"]["№ ОДПУ"].unique()

array(['89617'], dtype=object)

* В объекте ```г Уфа, ул. Даута Юлтыя, д.12``` подобъекты следует объединить - разделены несистемно:
  * ```г Уфа, ул. Даута Юлтыя, д.12```,
  * ```г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20654```
  * ```г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20655```

#### 2.5.5 ```г Уфа, ул. Комсомольская, д.15..```

In [36]:
s = 'г Уфа, ул. Комсомольская, д.15'
cond2_b = buildings["Адрес объекта"].str.match(s)
cond2_a = data["Адрес объекта"].str.match(s)

print(f"{data[cond1_a & cond2_a]['Адрес объекта'].unique()}\n")
buildings[cond1_b & cond2_b].iloc[:, :-1]

['г Уфа, ул. Комсомольская, д.15'
 'г Уфа, ул. Комсомольская, д.15, Подобъект №35951'
 'г Уфа, ул. Комсомольская, д.153'
 'г Уфа, ул. Комсомольская, д.153 корп.1'
 'г Уфа, ул. Комсомольская, д.157'
 'г Уфа, ул. Комсомольская, д.157 корп.1'
 'г Уфа, ул. Комсомольская, д.159'
 'г Уфа, ул. Комсомольская, д.159 корп.1'
 'г Уфа, ул. Комсомольская, д.15..']



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
2780,"г Уфа, ул. Комсомольская, д.15",Многоквартирный дом,23,2013-01-01,59310.4
2781,"г Уфа, ул. Комсомольская, д.153",Многоквартирный дом,9,1976-01-01,7698.9
2782,"г Уфа, ул. Комсомольская, д.153 корп.1",Многоквартирный дом,12,1982-01-01,4794.0
2784,"г Уфа, ул. Комсомольская, д.157",Многоквартирный дом,12,1957-01-01,4758.4
2785,"г Уфа, ул. Комсомольская, д.157 корп.1",Многоквартирный дом,9,1976-01-01,3830.4
2786,"г Уфа, ул. Комсомольская, д.159",Многоквартирный дом,9,1976-01-01,10843.5
2787,"г Уфа, ул. Комсомольская, д.159 корп.1",Многоквартирный дом,7,2005-01-01,15365.5


In [37]:
data[data["Адрес объекта"]=="г Уфа, ул. Комсомольская, д.15.."]

,index,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",Период потребления,Адрес объекта 2
41308,598,Уфа,224503,NaN,"г Уфа, ул. Комсомольская, д.15..",Многоквартирный дом,NaT,NaN,2022-08-01,"г Уфа, ул. Комсомольская, д.15"


* Объекта ```г Уфа, ул. Комсомольская, д.15..``` нет. Адрес объекта необходимо исправить.

### 2.6. Объекты без данных о выставленных счетах

In [38]:
addr_2 = sorted(
    list(set(buildings[cond1_b]["Адрес объекта 2"]) - set(data[cond1_a]["Адрес объекта 2"])))
df = buildings[cond1_b & buildings["Адрес объекта 2"].isin(addr_2)].iloc[:, :-1]
n = df.shape[0]
s = df['Общая площадь объекта'].sum() / 1e06
print(f"У {n} объектов общей площадью {s: 0.1f} млн. нет данных о выствленных счетах.")

save = False
if save:
  path = "/content/drive/MyDrive/ML_projects/08_anomaly_detection/data/01_raw/"
  file_name = "Тип строения, этажность, площадь, год постройки 2.xlsx"
  df.to_excel(f"{path}{file_name}")

У 752 объектов общей площадью  1.1 млн. нет данных о выствленных счетах.


### 2.7. Комментарии по пропускам в данных

1. У 25 многоквартирных домов Общая площадь объекта не превышает 0.1.

2. У 34 многоквартирных дома нет признака Дата постройки.

3. В базе многоквартирных домов 14 объектов с неуникальными адресами - по 2 на каждый адрес:
    * ```г Уфа, ул. Вологодская, д.13```,
    * ```г Уфа, ул. Вологодская, д.20```,
    * ```г Уфа, ул. Интернациональная, д.113```,
    * ```г Уфа, ул. Кирова, д.95```,
    * ```г Уфа, ул. Нежинская, д.6```,
    * ```г Уфа, ул. Революционная, д.88```,
    * ```г Уфа, ул. Энтузиастов, д.6```.

    На 3 объектах из 14 по адресам ниже не выставляются счета:
      * ```г Уфа, ул. Вологодская, д.20```,
      * ```г Уфа, ул. Интернациональная, д.113```,
      * ```г Уфа, ул. Нежинская, д.6```.

4. У подобъектов и объектов отличается ```№ ОДПУ```- учет ведется раздельно, при этом площадь - общая.
    *  Для использования в моделях необходимо включить подобъекты в объекты, при выдаче результата - оценить площадь подобъеков и объектов в разрезе ```№ ОДПУ```.

5. Многоквартирные дома, отсутствующие в базе данных объектов:
    * ```г Уфа, ул. Пархоменко, д.156 корп.Литер 2в``` в ```data``` не соответствует
    аналогу в ```buildings```:
      * ```г Уфа, ул. Пархоменко, д.156 корп.А```
      * ```г Уфа, ул. Пархоменко, д.156 корп.г```
      * ```г Уфа, ул. Пархоменко, д.156 корп.Д```
    * Объекта ```г Уфа, ул. Уфимское шоссе, д.4``` нет. Есть только ```г Уфа, ул. Уфимское шоссе, д.4 корп.1```, но у него другой ```№ ОДПУ```.
    * Объект ```г Уфа, ул. Владивостокская, д.10 корп.3,4``` в data должен попасть в ```г Уфа, ул. Владивостокская, д.10``` -  ```корп.3,4``` избыточен.
    * В объекте ```г Уфа, ул. Даута Юлтыя, д.12``` подобъекты следует объединить - разделены несистемно:
      * ```г Уфа, ул. Даута Юлтыя, д.12```,
      * ```г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20654```
      * ```г Уфа, ул. Даута Юлтыя, д.12, Подъезд №20655```
    * Объекта ```г Уфа, ул. Комсомольская, д.15..``` нет. Адрес объекта необходимо исправить.

6. У 752 объектов общей площадью  1.1 млн. нет данных о выствленных счетах.